In [17]:
import numpy as np
from func import *
from scipy.interpolate import interp1d

# 1. 
Derive the critical chi^2 values for which, that one can assign 68.3%, 90%, 95%, 95.4%, 99% and 99.73% confidence limits for degrees of freedom of 1, 2, 3, 4, 5 and 6 of the chi^2 distribution. Make a table for this.  

In [19]:
Chisq_integral(10, 5)

0.0752352461465122

In [ ]:
xarr = np.linspace(0, 100, 100)
yarr = Chisq_integral(xarr, 5)
f = interp1d(yarr, xarr, kind='linear', fill_value="extrapolate")

0.0012497305630312131